<a href="https://colab.research.google.com/github/anilsolanki2645/WebScarping/blob/main/IMDB_Movie_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scraping Multiples pages of IMDB at a time to fetch top 250 movies data sorted by Rating.

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* importing all the needed libraries

In [44]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import openpyxl
import matplotlib.pyplot as plt

* Declaring the variable of Headers

In [55]:
#Declaring the headers
headers = {"Accept-Language": "en-US,en;q=0.5"}
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
time=[]
rating=[]
metascore =[]
votes = []
description = []

In [56]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,250,250)

### collect and parse the information using
* requests lib and BeautifulSoup lib
* Store the information in variable
* and append in a list

In [ ]:
# Iterate through pages
for page in pages:
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
    print(page)
    # Make a request to IMDb
    page = requests.get("https://www.imdb.com/search/title/?sort=user_rating,desc&groups=top_1000&count=250", headers=headers)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(page.text, 'html.parser')

    # Find movie data containers
    movie_data = soup.findAll('div', {'class': 'sc-43986a27-0 gUQEVh'})

    # Pause to avoid overwhelming the server
    #sleep(randint(2, 8))

    # Print request status code for debugging
    #print("Request status code:", page_response.status_code)

    # Iterate through movie data
    for store in movie_data:

      # Extract information
      name = store.find('h3', class_='ipc-title__text').text
      movie_name.append(name)

      year_of_release = store.find('span', class_='sc-43986a27-8 jHYIIK dli-title-metadata-item').text
      year.append(year_of_release)

      runtime = store.findAll('span', class_='sc-43986a27-8 jHYIIK dli-title-metadata-item')[1].text
      time.append(runtime)

      # Handle the rating element gracefully
      rate = store.find('span', class_ = "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating").text.replace('\n', '').split('\xa0')[0]
      rating.append(rate)

      meta = store.find('span', class_ = "sc-b0901df4-0 bcQdDJ metacritic-score-box").text if store.find('span', class_ = "sc-b0901df4-0 bcQdDJ metacritic-score-box") else "*****"
      metascore.append(meta)

      #value = store.xpath('//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li[1]/div[1]/div/div/div[2]/div[2]/text()')
      value = store.findNext("div", class_ = "sc-53c98e73-0 kRnqtn").text.split('Votes')[-1]
      votes.append(value)

      # Description of the Movies -- Not explained in the Video, But you will figure it out.
      describe = store.findNext('div', class_ = 'ipc-html-content ipc-html-content--base sc-53c98e73-1 gMTWhH dli-plot-container').text
      description_ = describe.replace('\n', '') if len(describe) >1 else '*****'
      description.append(description_)



* creating a DataFrame

In [59]:
#creating a dataframe
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, "Votes" : votes, "Description": description  })
movie_list[1:103]

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Description
1,2. The Godfather,1972,2h 55m,9.2,100,"1,971,846","Don Vito Corleone, head of a mafia family, dec..."
2,3. The Dark Knight,2008,2h 32m,9.0,84,"2,810,373",When the menace known as the Joker wreaks havo...
3,4. Schindler's List,1993,3h 15m,9.0,95,"1,421,598","In German-occupied Poland during World War II,..."
4,5. The Lord of the Rings: The Return of the King,2003,3h 21m,9.0,94,"1,937,072",Gandalf and Aragorn lead the World of Men agai...
5,6. 12 Angry Men,1957,1h 36m,9.0,97,"843,811",The jury in a New York City murder trial is fr...
...,...,...,...,...,...,...,...
98,99. Eternal Sunshine of the Spotless Mind,2004,1h 48m,8.3,89,"1,056,314","When their relationship turns sour, a couple u..."
99,100. 2001: A Space Odyssey,1968,2h 29m,8.3,84,"705,594",After uncovering a mysterious artifact buried ...
100,101. Braveheart,1995,2h 58m,8.3,68,"1,076,357",Scottish warrior William Wallace leads his cou...
101,102. A Clockwork Orange,1971,2h 16m,8.3,77,"867,136","In the future, a sadistic gang leader is impri..."


In [60]:
movie_list[-3:]

,Movie Name,Year of Release,Watch Time,Movie Rating,Meatscore of movie,Votes,Description
247,248. Gran Torino,2008,1h 56m,8.1,72,"804,189",After a Hmong teenager tries to steal his priz...
248,"249. Lock, Stock and Two Smoking Barrels",1998,1h 47m,8.1,66,"607,252",Eddy persuades his three pals to pool money fo...
249,250. Warrior,2011,2h 20m,8.1,71,"491,057",The youngest son of an alcoholic former boxer ...


In [61]:
type(movie_list)

pandas.core.frame.DataFrame

* convert dataframe to Excel
* Create a Excel file

In [62]:
movie_list.to_excel('IMDB_Movie_Ratings.xlsx', header=True, index=False)